# Lecture 03 (2/2): Integer Programming
Let's consider an optimization problem where we want to form pairs of students for Assignment 1

## Constraints
We set a set of constraints and find an optimal formation of assignment pairs

In [34]:
import pulp
import numpy as np

In [35]:
students = np.arange(20)
students

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [36]:
from itertools import product
ss = list(product(students, students))
ss

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 13),
 (3, 14),
 (3, 15),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 18),
 (4, 19),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 

In [37]:
prob = pulp.LpProblem('AssignmentPair', pulp.LpMaximize)

pair = pulp.LpVariable.dicts('pair', ss, cat='Binary')

# Each student can only find one partner
for s in students:
    prob += pulp.lpSum([pair[s, s2] for s2 in students]) == 1

# The partnership needs to be symmetric
for _ss in ss:
    prob += pair[_ss[0], _ss[1]] - pair[_ss[1], _ss[0]] == 0

# Some students have already formed a pair
fixed_pairs = [(2, 3), (4, 5), (6, 7)]
for fp in fixed_pairs:
    prob += pair[fp[0], fp[1]] == 1


In [38]:
status = prob.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kotarohara/repo/Python/web-optimization/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/cf26dcc433a44595ab45648694c3dd3c-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/cf26dcc433a44595ab45648694c3dd3c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 429 COLUMNS
At line 2415 RHS
At line 2840 BOUNDS
At line 3242 ENDATA
Problem MODEL has 424 rows, 401 columns and 1164 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 13 rows, 91 columns (91 integer (91 of which binary)) and 169 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 1.5
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 0 iterations 8
Cb

'Optimal'

In [39]:
for key in ss:
    if pair[key].value() > 0:
        print(pair[key])
        print(pair[key].value())
        print()

pair_(0,_10)
1.0

pair_(1,_1)
1.0

pair_(2,_3)
1.0

pair_(3,_2)
1.0

pair_(4,_5)
1.0

pair_(5,_4)
1.0

pair_(6,_7)
1.0

pair_(7,_6)
1.0

pair_(8,_19)
1.0

pair_(9,_9)
1.0

pair_(10,_0)
1.0

pair_(11,_12)
1.0

pair_(12,_11)
1.0

pair_(13,_14)
1.0

pair_(14,_13)
1.0

pair_(15,_15)
1.0

pair_(16,_16)
1.0

pair_(17,_18)
1.0

pair_(18,_17)
1.0

pair_(19,_8)
1.0



## Objective
In addition to the constraints, let's add objective. Let's say we want to encourage a PhD student and a non-PhD student to work together.

In [69]:
students = [
    { "id": 0, "program": "master" },
    { "id": 1, "program": "master" },
    { "id": 2, "program": "master" },
    { "id": 3, "program": "master" },
    { "id": 4, "program": "master" },
    { "id": 5, "program": "master" },
    { "id": 6, "program": "master" },
    { "id": 7, "program": "master" },
    { "id": 8, "program": "master" },
    { "id": 9, "program": "master" },
    { "id": 10, "program": "phd" },
    { "id": 11, "program": "phd" },
    { "id": 12, "program": "phd" },
    { "id": 13, "program": "phd" },
    { "id": 14, "program": "phd" },
    { "id": 15, "program": "phd" },
    { "id": 16, "program": "phd" },
    { "id": 17, "program": "phd" },
    { "id": 18, "program": "phd" },
    { "id": 19, "program": "phd" },
]
sids = [s["id"] for s in students]
sids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [70]:
from itertools import product
ss = list(product(sids, sids))
ss

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 13),
 (3, 14),
 (3, 15),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 18),
 (4, 19),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 

In [71]:
prob = pulp.LpProblem('TeamMaking', pulp.LpMaximize)
pair = pulp.LpVariable.dicts('pair', ss, cat='Binary')

In [72]:
# Each student can only find one partner
for sid in sids:
    prob += pulp.lpSum([pair[sid, sid2] for sid2 in sids]) == 1

# The partnership needs to be symmetric
for _ss in ss:
    prob += pair[_ss[0], _ss[1]] - pair[_ss[1], _ss[0]] == 0

# Some students have already formed a pair
fixed_pairs = [(2, 3), (4, 5), (6, 7)]
for fp in fixed_pairs:
    prob += pair[fp[0], fp[1]] == 1

In [73]:
# Preference
def reward(sid1, sid2):
    reward = 0

    # It is nice if phd student and non-phd student work together
    program1 = students[sid1]['program']
    program2 = students[sid2]['program']
    if program1 != program2:
        reward += 1

    # It is nice if students work together with another student
    if sid1 != sid2:
        reward += 0.5

    return reward

In [74]:
prob += pulp.lpSum([pair[sid1, sid2] * reward(sid1, sid2) for sid1, sid2 in ss])

In [75]:
status = prob.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kotarohara/repo/Python/web-optimization/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/5788292eab9441e8ad9292ef50a0fdd5-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/5788292eab9441e8ad9292ef50a0fdd5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 428 COLUMNS
At line 2792 RHS
At line 3216 BOUNDS
At line 3617 ENDATA
Problem MODEL has 423 rows, 400 columns and 1163 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18 - 0.00 seconds
Cgl0004I processed model has 14 rows, 105 columns (105 integer (105 of which binary)) and 196 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solut

'Optimal'

In [76]:
for key in ss:
    if pair[key].value() > 0:
        print(pair[key])
        print(students[key[0]])
        print(students[key[1]])
        print()

pair_(0,_10)
{'id': 0, 'program': 'master'}
{'id': 10, 'program': 'phd'}

pair_(1,_13)
{'id': 1, 'program': 'master'}
{'id': 13, 'program': 'phd'}

pair_(2,_3)
{'id': 2, 'program': 'master'}
{'id': 3, 'program': 'master'}

pair_(3,_2)
{'id': 3, 'program': 'master'}
{'id': 2, 'program': 'master'}

pair_(4,_5)
{'id': 4, 'program': 'master'}
{'id': 5, 'program': 'master'}

pair_(5,_4)
{'id': 5, 'program': 'master'}
{'id': 4, 'program': 'master'}

pair_(6,_7)
{'id': 6, 'program': 'master'}
{'id': 7, 'program': 'master'}

pair_(7,_6)
{'id': 7, 'program': 'master'}
{'id': 6, 'program': 'master'}

pair_(8,_19)
{'id': 8, 'program': 'master'}
{'id': 19, 'program': 'phd'}

pair_(9,_12)
{'id': 9, 'program': 'master'}
{'id': 12, 'program': 'phd'}

pair_(10,_0)
{'id': 10, 'program': 'phd'}
{'id': 0, 'program': 'master'}

pair_(11,_18)
{'id': 11, 'program': 'phd'}
{'id': 18, 'program': 'phd'}

pair_(12,_9)
{'id': 12, 'program': 'phd'}
{'id': 9, 'program': 'master'}

pair_(13,_1)
{'id': 13, 'program'